In [22]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'indonesia-tourism-destination:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1481754%2F2448367%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240520%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240520T201426Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D82fae30f4cde59c46b0ed121eec08ab400a33ce0e79215e46b78ef6fb8a94998ba7b14b58558e544c6514299493c9c40401169784e19decbd07ed903a94f17d6b6ba61f371ddc0c14dd0449ad45b5b38c823bf9add5c1ab724a33a52ac26533d40c256e8faec07ee10241855d4d7a4835e86b25ca6d1ea8be8c12b1313c0f18590dfa450f34f0d87a3f86e7bf25d357de9686f27e8c465de8bf26d8d07417592470dcf3833ebd655f01cb2646baf992f734ca95191fecd8004abea943990ea5d985516f912001fd3def5132de9c5fce60909ba9ef5d7c96b952c357563b9c4a37d156a72cd625914fc2a87ee80771ca086231a47ae78ac8ace98ba81a26e1bd6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 162193 bytes downloaded
Downloaded and uncompressed: indonesia-tourism-destination
Data source import complete.


# Malang City Tourist Destination Recommendation System Based on Collaborative Filtering
<hr>

#### Source Dataset: https://www.kaggle.com/aprabowo/indonesia-tourism-destination
#### Oleh: [[Mikail](https://linkedin.com/in/mikailjordan), [Fander](https://https://linkedin.com/in/m-wahyu-elfander), [Salsa](https://linkedin.com/in/salsabila-azizah-abel)]

# 1. Data Preparation

## Import Library

In [23]:
# Untuk pengolahan data
import pandas as pd
import numpy as np
from zipfile import ZipFile
from pathlib import Path

# Untuk visualisasi data
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_palette('Set1')
sns.set()

# Untuk pemodelan
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Untuk menghilangkan warnings saat plotting seaborn
import warnings
warnings.filterwarnings('ignore')

# Untuk mengupload file
import os

## Preparing the Dataset used

In [24]:
# Upload semua dataset yang dipakai
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/indonesia-tourism-destination/package_tourism.csv
/kaggle/input/indonesia-tourism-destination/user.csv
/kaggle/input/indonesia-tourism-destination/tourism_rating.csv
/kaggle/input/indonesia-tourism-destination/tourism_with_id.csv


# 2. Data Understanding

## Prepare Data

In [25]:
# Menyimpan masing-masing dataset kedalam variabel

rating = pd.read_csv('/kaggle/input/indonesia-tourism-destination/tourism_rating.csv')
place = pd.read_csv('/kaggle/input/indonesia-tourism-destination/tourism_with_id.csv')
user = pd.read_csv('/kaggle/input/indonesia-tourism-destination/user.csv')

## Exploration of Features in Data

### Data Place

In [26]:
# Melihat gambaran data place

place.head(2)

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Unnamed: 11,Unnamed: 12
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,NaN,1
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,NaN,2


In [27]:
# Membuang kolom yang tidak dipakai

place = place.drop(['Unnamed: 11','Unnamed: 12'],axis=1)
place.head(2)

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125


In [28]:
# Merubah data agar hanya dari Kota Surabaya (Optional)

place = place[place['City']=='Surabaya']
place.head(2)

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long
391,392,Ekowisata Mangrove Wonorejo,Hutan Wisata Mangrove Surabaya merupakan wisat...,Cagar Alam,Surabaya,0,4.3,60.0,"{'lat': -7.308648199999999, 'lng': 112.8216622}",-7.308648,112.821662
392,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Cagar Alam,Surabaya,0,4.4,60.0,"{'lat': -7.2952211, 'lng': 112.8035603}",-7.295221,112.803560


In [29]:
place.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46 entries, 391 to 436
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Place_Id      46 non-null     int64  
 1   Place_Name    46 non-null     object 
 2   Description   46 non-null     object 
 3   Category      46 non-null     object 
 4   City          46 non-null     object 
 5   Price         46 non-null     int64  
 6   Rating        46 non-null     float64
 7   Time_Minutes  30 non-null     float64
 8   Coordinate    46 non-null     object 
 9   Lat           46 non-null     float64
 10  Long          46 non-null     float64
dtypes: float64(4), int64(2), object(5)
memory usage: 4.3+ KB


In [30]:
# Membuang kolom yang tidak dipakai

place = place.drop('Time_Minutes', axis=1)

### 3.2.2. Data rating

### Data User